In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from sklearn import preprocessing, pipeline, ensemble, compose
import datasets
import os

cols = ['age', 'workclass', 'fnlwgt', 'income']

real_path = '/hdd3/sonia/data/adult.csv'
dgpt2_path = '/hdd3/sonia/be_great/ckpts/dgpt2/adult-4col/samples.csv'
moe_path = '/hdd3/sonia/be_great/ckpts/moe/dgpt2/adult-4col/jul16/samples.csv'

/hdd2/sonia/miniconda3/envs/great/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
real = pd.read_csv(real_path)[cols]
dgpt2= pd.read_csv(dgpt2_path)
moe  = pd.read_csv(moe_path)

rs = 30
min_dataset_size = min(len(real), len(dgpt2), len(moe))
real = real.sample(min_dataset_size, random_state=rs)
dgpt2=dgpt2.sample(min_dataset_size, random_state=rs)
moe  =  moe.sample(min_dataset_size, random_state=rs)
real.shape, dgpt2.shape, moe.shape

((358, 4), (358, 4), (358, 4))

In [3]:
datadict = {'real':real, 'dgpt2':dgpt2, 'moe':moe,}
datadict.keys()

dict_keys(['real', 'dgpt2', 'moe'])

In [4]:
real['income'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [5]:
ords = ['workclass'] # MUST BE IN ORDER
nums = ['age', 'fnlwgt',]
labs = ['income']

# for each ord column, get all unique values occurign in real/synth, train/test
categories = []
for name in ords:
    s = set(real[name].unique().tolist())
    s.update(dgpt2[name].unique().tolist())
    s.update(moe[name].unique().tolist())
    
    categories.append( list(s) )
categories

ordenc = preprocessing.OrdinalEncoder(categories=categories)
numenc = preprocessing.StandardScaler()
lb = preprocessing.LabelBinarizer()

In [6]:
def create_pipeline(trainset):
    rfc = ensemble.RandomForestClassifier()
    preprocessing_pipeline = compose.ColumnTransformer([
        ("ordinal_preprocessor", ordenc, ords),
        ("numerical_preprocessor", numenc, nums),
    ])
    complete_pipeline = pipeline.Pipeline([
        ("preprocessor", preprocessing_pipeline),
        ("estimator", rfc)
    ])
    
    preprocessed_labels = lb.fit_transform(trainset[labs])
    complete_pipeline.fit(trainset[ords+nums], preprocessed_labels)
    return complete_pipeline

In [7]:
# %%capture
# create random forest pipelines
rfdict = {}
for src in datadict.keys():
    print(src)
    rfdict[src] = create_pipeline(datadict[src])

real
dgpt2
moe


/hdd2/sonia/miniconda3/envs/great/lib/python3.11/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/hdd2/sonia/miniconda3/envs/great/lib/python3.11/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/hdd2/sonia/miniconda3/envs/great/lib/python3.11/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [8]:
for data in datadict.keys():
    labels = lb.fit_transform(datadict[data][labs])
    for model in rfdict.keys():
        score = rfdict[model].score(datadict[data][ords+nums], labels)
        print(f'{model} on {data}: \t\t\t{score}')

real on real: 			1.0
dgpt2 on real: 			0.6396648044692738
moe on real: 			0.7374301675977654
real on dgpt2: 			0.6536312849162011
dgpt2 on dgpt2: 			1.0
moe on dgpt2: 			0.6396648044692738
real on moe: 			0.8184357541899442
dgpt2 on moe: 			0.7150837988826816
moe on moe: 			1.0
